<a href="https://colab.research.google.com/github/chetools/CHE4061_Fall2024/blob/main/MESHDistillation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -N -q https://raw.githubusercontent.com/chetools/chetools/main/tools/che5.ipynb -O che5.ipynb
!pip install importnb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 1.1 MB/s eta 0:00:00


In [2]:
from importnb import Notebook
with Notebook():
    from che5 import Props

import numpy as np
import jax
import jax.numpy as jnp
jax.config.update("jax_enable_x64", True)
from scipy.optimize import root_scalar
from scipy.optimize import root
from scipy.special import expit, logit
from plotly.subplots import make_subplots
from scipy.interpolate import Akima1DInterpolator

In [8]:
R=8.314
p=Props(['Ethanol', 'Water'])

In [4]:
def gamma(x,T):
    tau = p.NRTL_A + p.NRTL_B/T + p.NRTL_C*np.log(T) + p.NRTL_D*T
    G=np.exp(-p.NRTL_alpha*tau)
    xG = x@G
    xtauG_xG = (x@(tau*G))/xG
    return np.exp(xtauG_xG + x@((G*(tau - xtauG_xG[None,:])/xG[None,:]).T))

In [5]:
def dewP_ideal(y, T):
    P=1./(np.sum(y/p.Pvap(T)))
    return P, y*P/p.Pvap(T)

def dewT_ideal(y, P):

    def P_dev(T):
        return dewP_ideal(y, T)[0] - P
    T = root(P_dev, 300.).x[0]

    return T,  y*P/p.Pvap(T)

In [6]:
def bubbleP_NRTL(x, T):
    Pi= x*gamma(x,T)*p.Pvap(T)
    P=np.sum(Pi)
    return P, Pi/P

def bubbleT_NRTL(x, P):

    def f(T):
        return bubbleP_NRTL(x,T)[0]-P

    #mole-fraction weighted boiling points of each component at P
    #boiling points determined via Clausius Clapeyron, using the Hvap at the normal bp
    #for each component.  p.Hvap returns the heat of vaporization of all components for each
    #temperature if an array of temperatures is given.
    Tguess=np.dot(x,1/(1/p.Tbn-np.log(P/101325)*R/np.diagonal(p.Hvap(p.Tbn))))

    res=root_scalar(f, x0=Tguess, method='secant')
    if not(res.converged):
        return "FAIL", res
    T=root_scalar(f, x0=Tguess, method='secant').root
    Pi= x*gamma(x,T)*p.Pvap(T)
    P=np.sum(Pi)

    return T, Pi/P



In [9]:
P=101325
x1s=np.linspace(0,1,101)
Ts=[]
y1s=[]
for x1 in x1s:
    T, (y1,y2) = bubbleT_NRTL(np.array([x1, 1-x1]), P)
    Ts.append(T)
    y1s.append(y1)




In [10]:
def dewP_NRTL(y, T):
    def f(vec):
        P = vec[0]
        x = expit(vec[1:])  #Ensures that mole fractions are between 0 and 1

        fug_eqs = x*gamma(x,T) * p.Pvap(T)  - y*P
        xsum_eq = 1. - np.sum(x)

        return np.r_[fug_eqs, xsum_eq]

    #Assume ideal liquid dewP calculation for initial guess of P and liquid phase composition
    Pguess, xguess= dewP_ideal(z,T)

    #f (function to zero) maps values from -inf to inf, to values between 0 and 1
    #so xguess is mapped via logit which is the inverse function of expit
    v0 = np.r_[Pguess, logit(xguess)]
    res=root(f, v0)
    if not(res.success):
        return "FAILURE", res
    return res.x[0], expit(res.x[1:])

In [11]:
def dewT_NRTL(y, P):
    def f(vec):
        T = vec[0]
        x = expit(vec[1:])  #Ensures that mole fractions are between 0 and 1

        fug_eqs = x*gamma(x,T) * p.Pvap(T)  - y*P
        xsum_eq = 1. - np.sum(x)

        return np.r_[fug_eqs, xsum_eq]

    #Assume ideal liquid dewP calculation for initial guess of P and liquid phase composition
    Tguess, xguess= dewT_ideal(z,P)

    #f (function to zero) maps values from -inf to inf, to values between 0 and 1
    #so xguess is mapped via logit which is the inverse function of expit
    v0 = np.r_[Tguess, logit(xguess)]
    res=root(f, v0)
    if not(res.success):
        return "FAILURE", res
    return res.x[0], expit(res.x[1:])

In [12]:
def flash_idealPT(z, P, T):

    K=p.Pvap(T)/P
    def rachford(VF):
        return np.sum(z*(K-1)/(VF*(K-1) +1))

    res=root_scalar(rachford, bracket=(0,1))
    VF = res.root
    x=z/(1-VF + K *VF)
    y=K*x
    return x, y, VF

In [13]:
def flash_NRTL_PT(z, P, T, maxiter = 100, tol=1e-12):

    dewP, dewx = dewP_NRTL(z, T)
    bubbleP, bubbley = bubbleP_NRTL(z,T)

    xguess = (P-dewP)/(bubbleP-dewP) * (z - dewx) +  dewx

    for i in range(maxiter):
        K=gamma(xguess,T)*p.Pvap(T)/P

        def rachford(VF):
            return np.sum(z*(K-1)/(VF*(K-1) +1))

        res=root_scalar(rachford, bracket=(0,1))
        VF = res.root
        x=z/(1-VF + K *VF)
        if (np.linalg.norm(xguess-x)<tol):
            break
        xguess = x

    y=K*x
    return x, y, VF, i

In [14]:
Ftot = 2.
z = np.array([0.5,0.5])
P = 1e5


In [17]:
dewT, _ = dewT_NRTL(z, P)
bubbleT, _ = bubbleT_NRTL(z,P)

In [19]:
feedT = (bubbleT + dewT)/2

In [20]:
flash_NRTL_PT(z, P, feedT)

(Array([0.25722971, 0.74277029], dtype=float64),
 Array([0.56384623, 0.43615377], dtype=float64),
 0.791771709777732,
 62)